# 텍스트 생성 방법 : Transformers를 이용한 언어생성에 서로 다른 디코딩 방법 사용

In [15]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 개요
가장 두드러진 decoding 방법으로 Greedy search, Beam search, Top-K sampling, Top-p sampling 살펴보기

https://colab.research.google.com/drive/1yUGVmQ0nj8Hd3h0YV6PemQx0FtzpefGB?usp=sharing를 일부 수정하였습니다



Transformers를 설치하고 Model을 load하겠습니다.

이번 실습을 위해 SKT에서 공개한 KoGPT-2 모델을 사용해보도록 하겠습니다

In [17]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 1.6.14
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.2.0).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Git LFS initialized.
fatal: destination path 'kogpt2' already exists and is not an empty directory.


In [18]:
import torch
from tokenizers import SentencePieceBPETokenizer # 라이브러리에서 사용할 tokenizer를 import
from transformers import GPT2Config, GPT2LMHeadModel # transformers 라이브러리에서 사용할 언어 모델을 import

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")  # 입력 문장을 토큰 단위로 쪼개는 역할
tokenizer

Tokenizer(vocabulary_size=50000, model=SentencePieceBPE, unk_token=<unk>, replacement=▁, add_prefix_space=True, dropout=None)

In [19]:
config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config) # 쪼개진 입력 문장을 가지고 실제 문장생성을 진행
model

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

### **Greedy Search**

Greedy search는 단순히 가장 높은 확률을 가진 단어를 다음 단어로 선택합니다.   
$w_t = argmax_{w}P(w | w_{1:t-1})$ 는 각각의 timestep $t$ 입니다. 아래 그림은 greedy search을 보여줍니다.   

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

알고리즘은 단어 "The"에서 시작하여 다음 단어로 가장 높은 확률의 단어인 "nice" 등을 선택하는 탐욕법입니다. 그러므로 최종적으로 생성된 Word sequence는 "The", "nice", "woman"이며 전반적인 확률은 0.5x0.4 = 0.2로 계산됩니다.

다음 문맥 ("I", "enjoy", "walking", "with", "my", "cute", "dog")에서 GPT2를 사용하여 Word sequence를 생성할 수 있습니다.

##Transformers에서 greedy search를 사용하는 방법





In [20]:
# encode context the generation is conditioned on
def tokenizing(text):
    return torch.tensor(tokenizer.encode(text, add_special_tokens=False).ids).unsqueeze(0).to('cuda')

input_ids = tokenizing("이순신은 조선 중기의 무신이다.")

# 생성 모델은 generate 함수를 통해 다음 token을 생성 가능.
greedy_output = model.generate(   # 이 안에 들어가는 값들이 decoding 옵션들
        nput_ids, max_length=50) # generate text 는 context length를 포함해서 max_length에 도달하기 전까지 문장을 생성해낸다

print(tokenizer.decode(greedy_output.tolist()[0], skip_special_tokens=True))

이순신은 조선 중기의 무신이다.</s><s> 이 목록은 대한민국의 작곡가 겸 가수인 윤도현의 곡들을 모은 것이다.</s><s> 이 목록은 대한민국의 작곡가 겸 가수인 윤도현의 곡들을 모은 것이다.</s><s> 이 목록은 대한민국의 작곡가 겸 가수인


GPT2로 짧은 텍스트를 생성.   
생성된 단어 문맥은 합리적이지만 모델은 비슷한 단어를 반복하는 수준으로, 이러한 현상은 일반적인 언어생성 모델에서 나타나는 공통된 문제로 특히 Greedy search와 Beam search에서 훨씬 자주 나타난다

Greedy search의 주요 단점은 (그림에서 볼수 있듯이) 낮은 확률 단어 이후에 나올수 있는 더 높은 확률의 단어를 놓친다는 것이다.

예를 들면 단어 "has"는 0.9의 높은 조건부 확률을 가지고 있지만, 첫 검색단어중 두번째로 높은 조건부 확률 단어인 "dog" 이후에  숨어있는 형태이어서 Greedy search는 "The","dog","has"라는 Word sequence를 놓치게 된다.

이러한 문제는 Beam search에서 완화 가능하다.

### **Beam search**

Beam search는 각 Time step에서 가장 확률이 높은 Hypotheses의 num_beams를 유지하고 결국 전체 확률이 가장 높은 hypothesis를 선택하는 것으로 숨겨진 높은 확률 Word sequence를 놓칠 위험을 줄입니다.

`num_beams =2`라고 가정하고 Toy example을 보면

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

Time step=1일때, Beam search는 가장 가능성 높은 Hypothesis "The","woman"외에도 두번째로 가능성 높은 Hypothesis인 "The","dog"를 추적. 

Time step=2일때, Beam search는 Word sequence 확률 0.2를 가진 ("The","nice","woman") 보다 확률 0.36을 가진 ("The", "dog", "has")가 높다는 것을 찾습니다. 이것으로 Toy example에서 가장 가능성 높은 Word sequence를 발견 할 수 있다는 것을 보임.

Beam search는 항상 Greedy search보다 높은 확률의 결과 Sequence를 찾는 것이 가능. 그러나 이것이 가장 가능성 높은 결과를 찾은 것이라고는 보장할 수 없음. 


In [21]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,  
    max_length=50, 
    num_beams=5, 
    early_stopping=True
)

print(tokenizer.decode(beam_output.tolist()[0], skip_special_tokens=True))

이순신은 조선 중기의 무신이다.</s><s> 그 후, 조선이 멸망한 후, 조선을 건국하였다.</s><s> 그 후, 조선 건국 후, 조선을 건국하여 건국하였다.</s><s> 조선 건국 후, 조선을 건국하여 건국


동일한 Word sequence를 반복하는 문제.

 n-grams 패널티를 도입해, 이미 나타난 n-gram에 대해 다음 단어로 생성될 확률을 0으로 설정하여 두번 나타나지 않도록 함.

예를 들어, `no_repeat_ngram_size=2`을 설정한다면 2-gram이 두번 나타나는 것을 막을 수 있음.

In [22]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2,
    early_stopping=True
)

print(tokenizer.decode(beam_output.tolist()[0], skip_special_tokens=True))

이순신은 조선 중기의 무신이다.</s><s> 그 후, 그는 《삼국유사》(三國遺)의 편찬에 참여하였고, 《동국여지승람(東國地勝)》을 편찬하여 《삼국지


설정 결과, 더이상 반복이 나타나지 않음. 

이때 2-gram을 사용하게 될 경우 시의 이름이 전체 텍스트에서 한 번만 나타나기 때문에 city New York이 포함되어있는 기사에서는 사용하지 않는 것이 좋음

-> n-gram 패널티는 신중하게 사용해야 함

그 외에도, Beam search의 또 다른 중요한 특징은 생성된 Top beam을 비교하여 목적에 가장 적합한 Beam을 선택할 수 있다는 것.

Transformer에서 num_return_sequences 파라미터를 return 해야 하는 최대 num_beams 보다 작거나 같도록 설정. `num_return_sequences <= num_beams`로 설정된 코드를 확인 가능.

In [23]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output.tolist(), skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그는 《동국여지승람》(東國地勝)을 편찬하였고, 《조선왕조실록》(朝鮮王朝實錄)의 편찬에 참여하였다.
1: 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그는 《동국여지승람》(東國地勝)을 편찬하였고, 《조선왕조실록》(朝鮮王朝實錄)의 편찬을 도왔다.
2: 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그는 《동국여지승람》(東國地勝)을 편찬하였고, 《조선왕조실록》(朝鮮王朝實錄) 편찬에 참여하여 《삼국
3: 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그는 《동국여지승람》(東國地勝)을 편찬하였고, 《조선왕조실록》(朝鮮王朝實錄) 편찬에 참여하여 《대동
4: 이순신은 조선 중기의 무신이다.</s><s> 그 후, 그는 《동국여지승람》(東國地勝)을 편찬하였고, 《조선왕조실록》(朝鮮王朝實錄) 편찬에 참여하여 《조


코드 결과를 통해 볼수 있듯이 5개의 Beam hypotheses는 서로 약간 다름

최근, 개방형 생성에서는 Beam search가 최선의 선택사항이 아닐수 있는 몇 가지 이유가 제시되었다.

### **Sampling**

가장 기본적인 형태의 Sampling은 조건부 확률 분포에 따라 다음 단어 $w_t$를 무작위로 선택하는 것.


$$w_t \sim P(w|w_{1:t-1})$$


![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)


Sampling을 이용한 언어생성은 더이상 결정론적이지 않음. 단어 
$\text{"car"}$ 는 조건부확률 $P(w | \text{"The"})$에서 샘플링 된 후, $P(w | \text{"The"}, \text{"car"})$에서 $\text{"drives"}$를 샘플링 함.


In [24]:
# `transformers`에서 `do_sample=True`를 설정하고 `top_k=0`을 통해 *Top-K* sampling을 비활성화.
sample_output = model.generate(
    input_ids, 
    do_sample=True, # 완전 random sampling
    max_length=50, 
    top_k=0 # w/o top_k 추출
)

print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

이순신은 조선 중기의 무신이다.</s><s> 신성양우조금(新城兩神養)》으로 명칭이 바뀌었다.</s><s> 비슷한 시기에, 원의 작위를 받아 혼인하였다.</s><s> 아들이 원나라에서 홍종 인으로 태어났다.</s><s> 윤


결과는 일관성 없는 문장임
-> 이것은 sampling word sequences를 할때 모델이 일관성없이 횡설수설하는 문장을 발생시키는 큰 문제. ([Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751)).

이 때 적용할 수 있는 한가지 방법은 [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max). 의 이른바 `temperature`를 낮추어 분포 $P(w|w_{1:t-1})$를 더 선명하게 만드는 것. 
높은 확률의 단어의 가능성은 증가시키고 낮은 확률의 단어 가능성은 감소시키는 효과가 있음. 

temperature를 적용한다면 다음과 같은 그림을 보이게 됨.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

step=1의 다음 단어 분포는 더욱 선명해졌기 때문에 단어 $\text{"car"}$를 선택할 확률이 거의 없다

In [25]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=0, 
    temperature=0.7 # `temperature=0.7`를 설정하여 라이브러리에서 분포가 어떻게 변화했는지 보기

print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

이순신은 조선 중기의 무신이다.</s><s> 즉, "거기 윤이 없다"는 것이다.</s><s> 이는 '거기 윤'이 없다는 것이다.</s><s> 따라서, 독자들의 입장에 서 있는 독자들 또한 이 말을 번역해 내


이제 이상한 n-gram이 적고 출력 문장이 조금 더 일관성 있게 생성. 

temperature를 적용하면 분포가 덜 랜덤하지만 `temperature` $ \to 0$,을 설정한다면 temperature가 적용된 sampling은 greedy decoding과 같아지며 이전과 동일한 문제를 겪음.

### **Top-K Sampling**

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf) 

***Top-K*** sampling은 간단하지만 매우 강력한 생플링 방식을 도입했습니다. . *Top-K* sampling에서 가장 가능성 높은 다음 단어는 필터링 되고 확률 질량은 K 다음 단어에만 재분배됩니다. GPT2는 Top-K Sampling방식을 채택했는데, 이것이 Story Gerneration Task에 성공한 이유중 하나입니다.

Top-K Sampling을 더 잘 설명하기 위해 위의 예제에서 두 Sampling step에 사용되는 범위를 3단어에서 10단어로 확장합니다.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)


K=6을 설정하면 두 Sampling steps에서 Sampling pool을 6개의 단어로 제한합니다. $V_{\text{top-K}}$로 정의되는 가장 높은 6개의 단어로  sampling pool을 제한합니다.

첫 step에서 전체 확률 질량의 2/3인 0.68정도에 해당하는 단어에서 디코딩되지만, 두번째 step에서 거의 모든 확률질량인 0.99에서 디코딩합니다.

그럼에도 불구하고 그것이 두번째 sampling step에서 $\text{"not", "the", "small", "told"}$ 와 같은 다소 이상한 후보들을 성공적으로 제거가 가능했습니다.

In [26]:
# set top_k to 50
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=50
)

print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

이순신은 조선 중기의 무신이다.</s><s> 그러나, 이 책은 근대 이후 우리가 처한 모든 상황(물론)을 포괄하는 사상들-과 그 방법-을 보여주는 고전적인 사고방식-이 아니라, 실제 속에서 실천의 의미를 찾아가는 것이다.


### **Top-p (nucleus) sampling**

누적확률이 확률 p를 초과하는 최소한의 단어 집합에서 Sample을 추출.

그 후 확률 질량이 단어 집합 사이에 재분배. 이 방법은 다음 단어의 확률 분포에 따라 단어 집합의 크기가 동적으로 증가하거나 감소할 수 있음.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)



$p=0.92$을 설정할 경우, 상위 p Sample 추출은 $V_{\text{top-p}}$로 정의된 확률 질량의 $p=92\%$를 초과할 최소 단어 수를 선택.
첫번째 예에서 가장 가능성 높은 9개의 단어 ("nice", "dog", "car" ...  house)가 포함된 반면, 두번째 예에서는 상위 3개의 단어("drives", "is", "turns")만 선택해도 92%를 초과하게 됨. 
즉 높은 확률의 단어에만 Sampling 하고 그렇지 않은 단어는 Sampling할 확률이 매우 적게 됨.

Top-p는 또한 Top-K와 함께 사용될 수 있는데, 이것은 매우 낮은 순위의 단어를 피하면서도 일부 동적 선택을 허용함. 

In [29]:
# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=20, 
    top_p=0.92, 
    num_return_sequences=3 # 독립적으로 샘플링된 다중 출력을 얻기 위하여 파라미터를 다시 설정. `num_return_sequences > 1`: 
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output.tolist(), skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: 이순신은 조선 중기의 무신이다.</s><s> 이 전 대통령측도 같은 이유로 지난 2009년 3월 27일 사저로 이 전 대통령이 오셨을 당시와 지난 2009년 11월 28일 이 전 대통령으로부터 받은 사저에 대한 계약서
1: 이순신은 조선 중기의 무신이다.</s><s> 이 과정에서 "김씨가 자신의 재산을 노리고 있었다"는 의혹이 불거져 김 전 차관 측은 사실무근이라고 반박했습니다.</s><s> 김 전 차관이 김 전 차관에게 돈을 건넸다는 의혹에 대해 당시 수사팀은
2: 이순신은 조선 중기의 무신이다.</s><s> 그는 또한, 자신의 첫 번째 결혼 상대로 장가오리에게 "사랑해"라는 짧은 편지를 남겼다.</s><s> 그리고 결혼하여 두 아이를 낳았다.</s><s> 하지만 그 후, 그가 죽은 후, 그는


### **Appendix**

There are a couple of additional parameters for the `generate` method that were not mentioned above. We will explain them here briefly!

- `min_length` can be used to force the model to not produce an EOS token (= not finish the sentence) before `min_length` is reached. This is used quite frequently in summarization, but can be useful in general if the user wants to have longer outputs.
- `repetition_penalty` can be used to penalize words that were already generated or belong to the context. It was first introduced by [Kesker et al. (2019)](https://arxiv.org/abs/1909.05858) and is also used in the training objective in [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf). It can be quite effective at preventing repetitions, but seems to be very sensitive to different models and use cases, *e.g.* see this [discussion](https://github.com/huggingface/transformers/pull/2303) on Github.

- `attention_mask` can be used to mask padded tokens
- `pad_token_id`, `bos_token_id`, `eos_token_id`: If the model does not have those tokens by default, the user can manually choose other token ids to represent them.

For more information please also look into the `generate` function [docstring](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.TFPreTrainedModel.generate).